# Collecting the links

-----------
##### Loading packages

In [22]:
# requires to have selenium and  webdriver_manager installed

# Load packages
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import json
import time
import csv
import pandas as pd
import numpy as np

-----------
##### Navigating to bol.com

In [23]:
# Start Chrome
driver = webdriver.Chrome(ChromeDriverManager().install())

# navigating to bol.com
url = "https://www.bol.com/nl/nl/l/ebook-in-baby-en-peuterboeken/44325/61798+46052+46051+46050+45783+25561+8298+8296+8294+8293+8292+7419/?rating=all"
driver.get(url)
time.sleep(3)

C:\Users\Admin-HP\AppData\Local\Temp\ipykernel_4880\2494821642.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


-----------
##### Seed (links of the e-books) extraction

In [24]:
base_url = 'https://www.bol.com/nl/nl/l/ebook-in-baby-en-peuterboeken/44325/61798+46052+46051+46050+45783+25561+8298+8296+8294+8293+8292+7419/?page=1&rating=all'
url = base_url
driver.get(url)
time.sleep(1)
soup = BeautifulSoup(driver.page_source)

# extracting link from the first page
def get_url(base_url = 'https://www.bol.com/nl/nl/l/ebook-in-baby-en-peuterboeken/44325/61798+46052+46051+46050+45783+25561+8298+8296+8294+8293+8292+7419/?page=1&rating=all'):
    seed = []    
    for item in soup.find_all("a",{"data-test":"product-title"}):
        seed.append(item.get('href'))
    return(seed)  
links=get_url()
pages = soup.find_all("a",{"class":"js_pagination_item"})[-1].get_text()

print (str('There are'), int(pages) , str('navigation pages'))
print (str('Every page has'), len(links), str('e-books'))
print (str('Total e-books scraped'), len(links)*int(pages))

There are 46 navigation pages
Every page has 24 e-books
Total e-books scraped 1104


In [15]:
# function that iterates through pages 
def get_url(base_url = 'https://www.bol.com/nl/nl/l/ebook-in-baby-en-peuterboeken/44325/61798+46052+46051+46050+45783+25561+8298+8296+8294+8293+8292+7419/?page=1&rating=all',pages =20):
    url = base_url
    counter = 1
    driver.get(url)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source)
    seed = []
    for page in range(pages):
        for item in soup.find_all("a",{"data-test":"product-title"}):
            seed.append( 'https://www.bol.com' + str(item.get('href')))
        print('printing page '+str(counter))
        try:
            counter= counter + 1
            url= 'https://www.bol.com/nl/nl/l/ebook-in-baby-en-peuterboeken/44325/61798+46052+46051+46050+45783+25561+8298+8296+8294+8293+8292+7419/?page='+ str(counter)+'&rating=all'
            driver.get(url)
            time.sleep(1)
            soup = BeautifulSoup(driver.page_source)
            # something that block could be added
        except:
            break

    return(seed) 

In [16]:
base_url = 'https://www.bol.com/nl/nl/l/ebook-in-baby-en-peuterboeken/44325/61798+46052+46051+46050+45783+25561+8298+8296+8294+8293+8292+7419/?page=1&rating=all'
url = base_url
counter = 1
driver.get(url)
time.sleep(1)
soup = BeautifulSoup(driver.page_source)

# generating the seed
max_pages = int(soup.find_all("a",{"class":"js_pagination_item"})[-1].get_text())
links=get_url(pages = max_pages)

printing page 1
printing page 2
printing page 3
printing page 4
printing page 5
printing page 6
printing page 7
printing page 8
printing page 9
printing page 10
printing page 11
printing page 12
printing page 13
printing page 14
printing page 15
printing page 16
printing page 17
printing page 18
printing page 19
printing page 20
printing page 21
printing page 22
printing page 23
printing page 24
printing page 25
printing page 26
printing page 27
printing page 28
printing page 29
printing page 30
printing page 31
printing page 32
printing page 33
printing page 34
printing page 35
printing page 36
printing page 37
printing page 38
printing page 39
printing page 40
printing page 41
printing page 42
printing page 43
printing page 44
printing page 45


------------------ 
##### save the seeds in csv

In [8]:
header = ['e-book link']
with open('../data/seed.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for link in links:
        writer.writerows([[link]])
    

# Data collection

------------------ 
##### open the seeds in csv

In [3]:
links =[]
with open('../data/seed.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        links.append(row['e-book link'])


---------------
##### collecting data

this function extract
- title 
- Price (of the paper version)

In [6]:
# this chunk collects title and price of the paper version
links_sample=links[0:7]
data=[]
for link in links_sample:
    url = link
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    dictio= {'title':soup.find('span', {'data-test':"title"}).get_text(),
             'price':soup.find(class_='promo-price').get_text().replace('\n  ','.')[:-1]
            }
    data.append(dictio)
data

[{'title': 'Kikker en het vogeltje', 'price': '4.99'},
 {'title': 'Kikker in de kou', 'price': '4.99'},
 {'title': 'Het grote boek van Saskia en Jeroen', 'price': '4.99'},
 {'title': 'Minoes', 'price': '9.49'},
 {'title': 'Dikkie Dik -  Bundel blauw', 'price': '2.99'},
 {'title': 'Jip en Janneke er is er een jarig', 'price': '4.49'},
 {'title': 'Takkie', 'price': '4.49'}]

In [83]:
# summary regarding the price of the paper version
df = pd.DataFrame.from_dict(data)
clean_df=df.dropna(axis=0)
clean_df = clean_df.astype({'price':'float'})
print('The average is '+str(clean_df['price'].mean()))
print('The minimum is '+str(df['price'].min()))
print('The maximum is '+str(df['price'].max()))

The average is 6.4980000000000055
The minimum is 0.99
The maximum is 9.99


-------------------
The chunk below collects
- title
- price of the e-book
- number of pages (of the paper version)
- other existing versions of the book

In [21]:
links_sample=links[0:7]
data=[]
for link in links_sample:
    url = link
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    try:
        hardcover = soup.find_all("div",{"class":"feature-options feature-options--btns"})
        n_options=len(hardcover[0]('a'))
        #hard=hardcover[0]('a')[1].get('href')

        for option in soup.find_all("div",{"class":"feature-options feature-options--btns"})[0]('a'):
            if 'hard' in str(option):
                hardcover_link = 'https://www.bol.com' + str(option.get('href'))
                driver.get(hardcover_link)
                soup = BeautifulSoup(driver.page_source)
                for info_bar_hard in soup.find_all("ul",{"class":"product-small-specs product-small-specs--large"})[0]('li'):
                    if 'pagin' in info_bar_hard.get_text():
                        number_pager_hardcover = float(info_bar_hard.get_text().replace(" pagina's", '').replace("\n", '').strip())
                    else:
                        0
            else :
                number_pager_hardcover= 'no hardcover'
    except:
        number_pager_hardcover= 'no hardcover'
        n_options = 'no other options'

    dictio= {'title':soup.find('span', {'data-test':"title"}).get_text(),
             'price':float(soup.find(class_='promo-price').get_text().replace('\n  ','.')[:-1]),
             'purchase options': n_options,
             'number of pages': number_pager_hardcover
            }
    data.append(dictio)
data

[{'title': 'Kikker en het vogeltje',
  'price': 16.99,
  'purchase options': 3,
  'number of pages': 32.0},
 {'title': 'Kikker in de kou',
  'price': 16.99,
  'purchase options': 3,
  'number of pages': 32.0},
 {'title': 'Het grote boek van Saskia en Jeroen',
  'price': 4.99,
  'purchase options': 2,
  'number of pages': 'no hardcover'},
 {'title': 'Minoes',
  'price': 21.98,
  'purchase options': 5,
  'number of pages': 155.0},
 {'title': 'Dikkie Dik -  Bundel blauw',
  'price': 2.99,
  'purchase options': 2,
  'number of pages': 'no hardcover'},
 {'title': 'Jip en Janneke er is er een jarig',
  'price': 8.99,
  'purchase options': 2,
  'number of pages': 32.0},
 {'title': 'Takkie',
  'price': 9.99,
  'purchase options': 2,
  'number of pages': 48.0}]

-------------------
The chunk below collects
- title
- price of the e-book
- number of pages (of the hardcover version)
- number of pages (of the paper back version)
- other existing versions of the book

In [10]:
links_sample=links[0:9]
data=[]
for link in links_sample:
    url = link
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    price_ebook = float(soup.find(class_='promo-price').get_text().replace('\n  ','.').replace('-','00')[:-1])
    title = soup.find('span', {'data-test':"title"}).get_text()
    try:
        hardcover = soup.find_all("div",{"class":"feature-options feature-options--btns"})
        n_options=len(hardcover[0]('a'))
        #hard=hardcover[0]('a')[1].get('href')

        for option in soup.find_all("div",{"class":"feature-options feature-options--btns"})[0]('a'):
            if 'ardcover' in str(option):
                hardcover_link = 'https://www.bol.com' + str(option.get('href'))
                driver.get(hardcover_link)
                soup = BeautifulSoup(driver.page_source)
                for info_bar_hard in soup.find_all("ul",{"class":"product-small-specs product-small-specs--large"})[0]('li'):
                    if 'pagin' in info_bar_hard.get_text():
                        number_pages_hardcover = int(info_bar_hard.get_text().replace(" pagina's", '').replace("\n", '').strip())
                    else:
                        0
            elif 'aperback' in str(option):
                paperback_link = 'https://www.bol.com' + str(option.get('href'))
                driver.get(paperback_link)
                soup = BeautifulSoup(driver.page_source)
                for info_bar_paper in soup.find_all("ul",{"class":"product-small-specs product-small-specs--large"})[0]('li'):
                    if 'pagin' in info_bar_paper.get_text():
                        number_pages_paperback = int(info_bar_paper.get_text().replace(" pagina's", '').replace("\n", '').strip())
                    else:
                        0
            else:
                number_pages_paperback = 'no paperback'
                number_pages_hardcover = 'no hardcover'
    except:
        number_pages_hardcover= 'no hardcover'
        number_pages_paperback = 'no paperback'
        n_options = 'no other options'
    

    dictio= {'title': title,
             'price ebook': price_ebook,
             'purchase options': n_options,
             'number of pages (hardcover)': number_pages_hardcover,
             'number of pages (paperback)' : number_pages_paperback
            }
    data.append(dictio)
    print (str(link)+' extracted')
print(len(data),str('books scraped'))

https://www.bol.com/nl/nl/p/kikker-en-het-vogeltje/9200000022229587/ extracted
https://www.bol.com/nl/nl/p/kikker-in-de-kou/9200000022076749/ extracted
https://www.bol.com/nl/nl/p/het-grote-boek-van-saskia-en-jeroen/9200000027131643/ extracted
https://www.bol.com/nl/nl/p/minoes/9200000009952620/ extracted
https://www.bol.com/nl/nl/p/dikkie-dik-bundel-blauw/9200000015498008/ extracted
https://www.bol.com/nl/nl/p/jip-en-janneke-er-is-er-een-jarig/9200000011127429/ extracted
https://www.bol.com/nl/nl/p/takkie/9200000049684680/ extracted
https://www.bol.com/nl/nl/p/de-voorleesboot/9200000084637836/ extracted
https://www.bol.com/nl/nl/p/filou-pummel/9200000034019919/ extracted
9 books scraped


---------
##### most updated scraper
This code extract
- title
- price of the e-book
- number of pages (of the hardcover version)
- number of pages (of the paper back version)
- other existing versions of the book
- ratings
- language of the book

In [15]:
data=[]
for link in links:
    url = link
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    price_ebook = float(soup.find(class_='promo-price').get_text().replace('\n  ','.').replace('-','00')[:-1])
    title = soup.find('span', {'data-test':"title"}).get_text()
    try:
        language = soup.find('a', {'data-analytics-tag':"language"}).text.replace('Taal:', '').strip()
    except:
        language = 'no language'
    for row in soup.find_all(class_= "specs__row"):
        if 'Leeftijd' in row.get_text(): #Leeftijd means age in dutch
            age = row.find(class_= "specs__value").get_text().replace('\n','').replace('                      ','').strip()
        else:
            0
    try:
        hardcover = soup.find_all("div",{"class":"feature-options feature-options--btns"})
        n_options=len(hardcover[0]('a'))
        #hard=hardcover[0]('a')[1].get('href')

        for option in soup.find_all("div",{"class":"feature-options feature-options--btns"})[0]('a'):
            if 'ardcover' in str(option):
                hardcover_link = 'https://www.bol.com' + str(option.get('href'))
                driver.get(hardcover_link)
                soup = BeautifulSoup(driver.page_source)
                for info_bar_hard in soup.find_all("ul",{"class":"product-small-specs product-small-specs--large"})[0]('li'):
                    if 'pagin' in info_bar_hard.get_text():
                        number_pages_hardcover = int(info_bar_hard.get_text().replace(" pagina's", '').replace("\n", '').strip())
                    else:
                        0
            elif 'aperback' in str(option):
                paperback_link = 'https://www.bol.com' + str(option.get('href'))
                driver.get(paperback_link)
                soup = BeautifulSoup(driver.page_source)
                for info_bar_paper in soup.find_all("ul",{"class":"product-small-specs product-small-specs--large"})[0]('li'):
                    if 'pagin' in info_bar_paper.get_text():
                        number_pages_paperback = int(info_bar_paper.get_text().replace(" pagina's", '').replace("\n", '').strip())
                    else:
                        0
            else:
                number_pages_paperback = 'no paperback'
                number_pages_hardcover = 'no hardcover'
    except:
        number_pages_hardcover= 'no hardcover'
        number_pages_paperback = 'no paperback'
        n_options = 'no other options'
    try:
        reviews = soup.find('div', {'data-test':"rating-suffix"}).get_text()
    except:
        reviews = 'no reviews'
        
        
    

    dictio= {'title': title,
             'price ebook': price_ebook,
             'purchase options': n_options,
             'number of pages (hardcover)': number_pages_hardcover,
             'number of pages (paperback)' : number_pages_paperback,
             'reviews': reviews,
             'language':language,
             'age': age
            }
    data.append(dictio)
    print (str(link)+' extracted')
print(len(data),str('books scraped'))
data

https://www.bol.com/nl/nl/p/kikker-en-het-vogeltje/9200000022229587/ extracted
https://www.bol.com/nl/nl/p/kikker-in-de-kou/9200000022076749/ extracted
https://www.bol.com/nl/nl/p/het-grote-boek-van-saskia-en-jeroen/9200000027131643/ extracted
https://www.bol.com/nl/nl/p/minoes/9200000009952620/ extracted
https://www.bol.com/nl/nl/p/dikkie-dik-bundel-blauw/9200000015498008/ extracted
https://www.bol.com/nl/nl/p/jip-en-janneke-er-is-er-een-jarig/9200000011127429/ extracted
https://www.bol.com/nl/nl/p/takkie/9200000049684680/ extracted
https://www.bol.com/nl/nl/p/de-voorleesboot/9200000084637836/ extracted
https://www.bol.com/nl/nl/p/filou-pummel/9200000034019919/ extracted
https://www.bol.com/nl/nl/p/dikkie-dik-coleccion-naranja/9200000028780681/ extracted
https://www.bol.com/nl/nl/p/jip-en-janneke-spelen-buiten/9200000011087227/ extracted
https://www.bol.com/nl/nl/p/woezel-pip-geheimen-van-de-tovertuin/9300000005487917/ extracted
https://www.bol.com/nl/nl/p/woezel-pip-land-van-je-ogen-

https://www.bol.com/nl/nl/p/bing-bing-hide-and-seek/9200000041422946/ extracted
https://www.bol.com/nl/nl/p/babybooks-hay-um-pato-em-el-trafico/9200000058735271/ extracted
https://www.bol.com/nl/nl/p/little-golden-book-richard-scarry-s-cars-and-trucks/9200000066823221/ extracted
https://www.bol.com/nl/nl/p/al-pha-s-bet/9200000033781074/ extracted
https://www.bol.com/nl/nl/p/honey/9200000076961142/ extracted
https://www.bol.com/nl/nl/p/estaba-la-rana/9200000056506514/ extracted
https://www.bol.com/nl/nl/p/ladybird-christmas-carols/9200000035702063/ extracted
https://www.bol.com/nl/nl/p/backseat-a-b-see/9200000033101499/ extracted
https://www.bol.com/nl/nl/p/l-anniversaire-de-liselotte/9200000034384848/ extracted
https://www.bol.com/nl/nl/p/food-truck-fest/9200000090134567/ extracted
https://www.bol.com/nl/nl/p/curious-baby-curious-george-curious-baby-my-curious-dreamer/9200000020060617/ extracted
https://www.bol.com/nl/nl/p/choo-choo-clickety-clack/9200000033848014/ extracted
https://ww

https://www.bol.com/nl/nl/p/demolicion/9200000082844385/ extracted
https://www.bol.com/nl/nl/p/duck-goose-duck-goose-it-s-time-for-christmas-read-listen-edition/9200000089105514/ extracted
https://www.bol.com/nl/nl/p/stanley-stanley-s-train/9200000105751004/ extracted
https://www.bol.com/nl/nl/p/look-look/9200000058890165/ extracted
https://www.bol.com/nl/nl/p/god-made-you-nose-to-toes/9200000072967690/ extracted
https://www.bol.com/nl/nl/p/elisa-va-a-cursosde-natacion/9200000033983482/ extracted
https://www.bol.com/nl/nl/p/bambi/9200000063434738/ extracted
https://www.bol.com/nl/nl/p/demolition/9300000039272042/ extracted
https://www.bol.com/nl/nl/p/hamsters-holding-hands/9200000032981431/ extracted
https://www.bol.com/nl/nl/p/el-cumpleanos-de-elisa/9200000033912224/ extracted
https://www.bol.com/nl/nl/p/the-abc-character-builder/9300000093328402/ extracted
https://www.bol.com/nl/nl/p/aquarium/9300000117483835/ extracted
https://www.bol.com/nl/nl/p/alice-s-adventures-in-wonderland/920

https://www.bol.com/nl/nl/p/one-piece-of-string/9200000079290950/ extracted
https://www.bol.com/nl/nl/p/one-to-ten-animal-mayhem/9200000022972058/ extracted
https://www.bol.com/nl/nl/p/spuk-im-reihenhaus/9200000033984851/ extracted
https://www.bol.com/nl/nl/p/a-is-for-apex/9200000095445920/ extracted
https://www.bol.com/nl/nl/p/veggies-with-wedgies-present-doin-the-wedgie/9200000031905094/ extracted
https://www.bol.com/nl/nl/p/little-blossom-stories-the-case-of-the-lost-pup/9200000104580421/ extracted
https://www.bol.com/nl/nl/p/rabbits-in-the-snow-a-book-of-opposites/9200000045768737/ extracted
https://www.bol.com/nl/nl/p/bob-books-rhyming-words/9200000037713699/ extracted
https://www.bol.com/nl/nl/p/if-you-give-merry-christmas-mouse/9300000093209172/ extracted
https://www.bol.com/nl/nl/p/little-golden-book-where-do-giggles-come-from/9200000017845631/ extracted
https://www.bol.com/nl/nl/p/little-blossom-stories-the-case-of-the-lost-hen/9200000104580541/ extracted
https://www.bol.com/n

https://www.bol.com/nl/nl/p/in-over-and-on-the-farm/9200000035295511/ extracted
https://www.bol.com/nl/nl/p/tomo-s-adventure-series-tomo-explores-the-world/9200000065983863/ extracted
https://www.bol.com/nl/nl/p/bear-sees-colors/9200000026921691/ extracted
https://www.bol.com/nl/nl/p/anne-of-green-gables-anne-s-numbers/9200000087479639/ extracted
https://www.bol.com/nl/nl/p/the-royal-tea-party/1001004011805081/ extracted
https://www.bol.com/nl/nl/p/ready-for-series-ready-for-autumn/9200000034059437/ extracted
https://www.bol.com/nl/nl/p/kleiner-ritter-milchling/9200000034228140/ extracted
https://www.bol.com/nl/nl/p/lesemaus-max-und-der-wackelzahn/9200000055585571/ extracted
https://www.bol.com/nl/nl/p/kids-from-a-z/9200000035595253/ extracted
https://www.bol.com/nl/nl/p/anne-of-green-gables-anne-s-feelings/9200000097566521/ extracted
https://www.bol.com/nl/nl/p/my-bike/9200000085233086/ extracted
https://www.bol.com/nl/nl/p/bugs/9200000047685931/ extracted
https://www.bol.com/nl/nl/p/

...

https://www.bol.com/nl/nl/p/animal-antics-a-to-z-walter-warthog-s-wonderful-wagon/9300000039385993/ extracted
https://www.bol.com/nl/nl/p/plumps/9200000033607971/ extracted
https://www.bol.com/nl/nl/p/bright-early-books-ten-eggs-in-a-nest/9200000024832425/ extracted
https://www.bol.com/nl/nl/p/love-you-hug-you-read-to-you/9200000042961449/ extracted
https://www.bol.com/nl/nl/p/the-tale-of-two-bad-mice/9300000080562934/ extracted
https://www.bol.com/nl/nl/p/mommy-hugs/9200000000413824/ extracted
https://www.bol.com/nl/nl/p/teddy-bear-tears/9200000035113640/ extracted
https://www.bol.com/nl/nl/p/die-schonsten-marchen-von-hans-christian-andersen-das-feuerzeug/9300000075952203/ extracted
https://www.bol.com/nl/nl/p/bart-en-maartje/9200000015035376/ extracted
https://www.bol.com/nl/nl/p/c-est-chouette-d-etre-amis/9200000034026656/ extracted
https://www.bol.com/nl/nl/p/awesome-engines-big-digger-abc/9200000048011727/ extracted
https://www.bol.com/nl/nl/p/donald-el-camion/9200000093221716/ ex

https://www.bol.com/nl/nl/p/little-golden-book-the-poky-little-puppy-s-wonderful-winter-day/9200000082960633/ extracted
https://www.bol.com/nl/nl/p/rumble-grumble-hush/9200000089106946/ extracted
https://www.bol.com/nl/nl/p/peek-a-little-boo/9200000034180242/ extracted
https://www.bol.com/nl/nl/p/abcs-on-wings/9200000035549494/ extracted
https://www.bol.com/nl/nl/p/apple-farmer-annie-board-book/9200000040392545/ extracted
https://www.bol.com/nl/nl/p/the-tale-of-two-bad-mice/9200000028577883/ extracted
https://www.bol.com/nl/nl/p/einschlafen-mit-laura-1-lauras-stern-gutenacht-geschichten/9200000040989181/ extracted
https://www.bol.com/nl/nl/p/big-girl-panties/9200000033013711/ extracted
https://www.bol.com/nl/nl/p/garcia-colette-go-exploring/9200000065856685/ extracted
https://www.bol.com/nl/nl/p/baby-max-and-ruby-counting-peas/9200000042798503/ extracted
https://www.bol.com/nl/nl/p/wee-william/9200000035558531/ extracted
https://www.bol.com/nl/nl/p/math-is-fun-splitting-the-herd/920000

https://www.bol.com/nl/nl/p/forever-or-a-day/9200000086627071/ extracted
https://www.bol.com/nl/nl/p/schmetterlinge-wohin-seid-ihr-geflogen/9200000090082342/ extracted
https://www.bol.com/nl/nl/p/the-seashore-book/9200000076499254/ extracted
https://www.bol.com/nl/nl/p/mr-men-and-little-miss-mr-messy/9200000033994326/ extracted
https://www.bol.com/nl/nl/p/beach-day/9200000014336141/ extracted
https://www.bol.com/nl/nl/p/shh-bears-sleeping/9200000041743366/ extracted
https://www.bol.com/nl/nl/p/two-by-two/9200000091367142/ extracted
https://www.bol.com/nl/nl/p/math-is-fun-splitting-the-herd/9200000099197897/ extracted
https://www.bol.com/nl/nl/p/emelia-and-alex-s-day-in-the-yard/9200000078669131/ extracted
https://www.bol.com/nl/nl/p/nancy-carlson-picture-books-think-big/9200000033746446/ extracted
https://www.bol.com/nl/nl/p/ladybug-girl-ladybug-girl-feels-happy/9200000034347984/ extracted
https://www.bol.com/nl/nl/p/cgtv-reader-curious-george-lemonade-stand/9200000053068707/ extracted

https://www.bol.com/nl/nl/p/mollie-the-cat/9200000078669113/ extracted
https://www.bol.com/nl/nl/p/rooster-s-day-on-the-farm/9200000105310968/ extracted
https://www.bol.com/nl/nl/p/sassy-baby-loves-shapes/9200000034571735/ extracted
https://www.bol.com/nl/nl/p/monday-super-fun-day/9300000035013966/ extracted
https://www.bol.com/nl/nl/p/train/9200000041210379/ extracted
https://www.bol.com/nl/nl/p/der-kleine-hase-munk-weihnachten/9200000034192533/ extracted
1077 books scraped


[{'title': 'Kikker en het vogeltje',
  'price ebook': 4.99,
  'purchase options': 3,
  'number of pages (hardcover)': 32,
  'number of pages (paperback)': 'no paperback',
  'reviews': '4,7/5 (54 reviews)',
  'language': 'Nederlands',
  'age': 'vanaf 3 jaar'},
 {'title': 'Kikker in de kou',
  'price ebook': 4.99,
  'purchase options': 3,
  'number of pages (hardcover)': 32,
  'number of pages (paperback)': 'no paperback',
  'reviews': '4,6/5 (14 reviews)',
  'language': 'Nederlands',
  'age': 'vanaf 3 jaar'},
 {'title': 'Het grote boek van Saskia en Jeroen',
  'price ebook': 4.99,
  'purchase options': 3,
  'number of pages (hardcover)': 224,
  'number of pages (paperback)': 'no paperback',
  'reviews': '4,3/5 (6 reviews)',
  'language': 'Nederlands',
  'age': 'vanaf 3 jaar, vanaf 4 jaar, vanaf 5 jaar, vanaf 6 jaar'},
 {'title': 'Minoes',
  'price ebook': 9.49,
  'purchase options': 5,
  'number of pages (hardcover)': 155,
  'number of pages (paperback)': 160,
  'reviews': '4,8/5 (30 re

In [14]:
data

[{'title': 'Kikker en het vogeltje',
  'price ebook': 4.99,
  'purchase options': 3,
  'number of pages (hardcover)': 32,
  'number of pages (paperback)': 'no paperback',
  'reviews': '4,7/5 (54 reviews)',
  'language': 'Nederlands',
  'age': 'vanaf 3 jaar'},
 {'title': 'Kikker in de kou',
  'price ebook': 4.99,
  'purchase options': 3,
  'number of pages (hardcover)': 32,
  'number of pages (paperback)': 'no paperback',
  'reviews': '4,6/5 (14 reviews)',
  'language': 'Nederlands',
  'age': 'vanaf 3 jaar'},
 {'title': 'Het grote boek van Saskia en Jeroen',
  'price ebook': 4.99,
  'purchase options': 3,
  'number of pages (hardcover)': 224,
  'number of pages (paperback)': 'no paperback',
  'reviews': '4,3/5 (6 reviews)',
  'language': 'Nederlands',
  'age': 'vanaf 3 jaar, vanaf 4 jaar, vanaf 5 jaar, vanaf 6 jaar'},
 {'title': 'Minoes',
  'price ebook': 9.49,
  'purchase options': 5,
  'number of pages (hardcover)': 155,
  'number of pages (paperback)': 160,
  'reviews': '4,8/5 (30 re

In [20]:
len(data)

1077

----------
##### saving output in json

In [21]:
with open("../data/dataset", "w") as f:
    json.dump(data, f, indent=4)